In [1]:
import random

class Propaganda:
    NEGATIVE = "NEGATIVE"
    POSITIVE = "POSITIVE"

class Review:
    def __init__(self,sentence,SUBJprop):
        self.sentence = sentence
        self.SUBJprop = SUBJprop
        self.propaganda = self.get_propaganda()        

    def get_propaganda(self):
        if int(self.SUBJprop) <=3 :
            return Propaganda.NEGATIVE
        else:
            return Propaganda.POSITIVE


class ReviewContainer:
    def __init__(self,reviews):
        self.reviews = reviews

    def get_sentence(self):
        return [x.sentence for x  in self.reviews]

    def get_propaganda(self):
        return [x.propaganda for x in self.reviews]

    def evenly_distribute(self):
        negative = list(filter(lambda x: x.propaganda == Propaganda.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.propaganda == Propaganda.POSITIVE, self.reviews))
        negative_shrunk = negative[:len(positive)]
        self.reviews = positive + negative_shrunk
        random.shuffle(self.reviews)

In [2]:
import pandas as pd
import numpy as np
import nltk
import string
from string import digits

import nltk


reviews = []
data  = pd.read_excel('Data/finalDataset.xlsx', engine='openpyxl')
df = pd.DataFrame(data.astype(str) , columns = ['Sentence','SUBJprop'])
# iterate elements of attribute "Sentence" and "SUBJprop" and push to the array "reviews"

for index, row in df.iterrows():
    sentence = row['Sentence']
    prop = row['SUBJprop']
    reviews.append(Review(sentence,prop))


print("Total Rows:")
print(len(reviews))
print("Total Positive:")
print(len(list(filter(lambda x: x.propaganda == Propaganda.POSITIVE, reviews))))
print("Total Negative:")
print(len(list(filter(lambda x: x.propaganda == Propaganda.NEGATIVE, reviews))))



Total Rows:
16774
Total Positive:
3780
Total Negative:
12994


In [3]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import string
punct = string.punctuation

stopwords = list(STOP_WORDS)


nlp = spacy.load('en_core_web_sm')

def text_data_cleaning(sentence):
    doc = nlp(sentence)
    
    tokens = []
    for token in doc:
        if token.lemma_ != "-PRON-":
            temp = token.lemma_.lower().strip()
        else:
            temp = token.lower_
        tokens.append(temp)
    
    cleaned_tokens = []
    for token in tokens:
        if token not in stopwords and token not in punct:
            cleaned_tokens.append(token)
    return cleaned_tokens


In [ ]:
text_data_cleaning("    Hello how are you. Like this video")


In [4]:
from sklearn.model_selection import train_test_split

neg_prop = list(filter(lambda x: x.propaganda == Propaganda.NEGATIVE, reviews))
pos_prop = list(filter(lambda x: x.propaganda == Propaganda.POSITIVE, reviews))

########################################################################################
#split trainig and DevTest dataset
neg_train, neg_devtest  = train_test_split(neg_prop , train_size=0.7, random_state = 42 )
pos_train, pos_devtest = train_test_split(pos_prop , train_size=0.7, random_state = 42 )
########################################################################################
#prepare training dataset
train = neg_train + pos_train
random.shuffle(train)
########################################################################################
#prepare development and test dataset
neg_dev, neg_test = train_test_split(neg_devtest , train_size=0.5, random_state = 42 )
pos_dev, pos_test = train_test_split(pos_devtest , train_size=0.5, random_state = 42 )

dev = neg_dev + pos_dev
random.shuffle(dev)

test = neg_test + pos_test
random.shuffle(test)
########################################################################################
print("Total Train:")
print(len(train))
print("Positive:")
print(len(pos_train))
print("Negative:")
print(len(neg_train))

print("\nTotal Development:")
print(len(dev))
print("Positive:")
print(len(pos_dev))
print("Negative:")
print(len(neg_dev))

print("\nTotal Test:")
print(len(test))
print("Positive:")
print(len(pos_test))
print("Negative:")
print(len(neg_test))

Total Train:
11741
Positive:
2646
Negative:
9095

Total Development:
2516
Positive:
567
Negative:
1949

Total Test:
2517
Positive:
567
Negative:
1950


In [5]:
train_container = ReviewContainer(train)
train_container.evenly_distribute()

train_sentence = train_container.get_sentence()   
train_propaganda = train_container.get_propaganda() 

development_sentence = [x.sentence for x in dev]
development_propaganda = [x.propaganda for x in dev]

test_sentence = [x.sentence for x in test]
test_propaganda = [x.propaganda for x in test]

print("Total rows after balance training dataset:")
print(len(train_sentence))

print("Positive Propaganda:")
print(train_propaganda.count(Propaganda.POSITIVE))

print("Negative Propaganda:")
print(train_propaganda.count(Propaganda.NEGATIVE))


Total rows after balance training dataset:
5292
Positive Propaganda:
2646
Negative Propaganda:
2646


In [6]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [9]:
tfidf = TfidfVectorizer(tokenizer = text_data_cleaning)
classifier = LinearSVC()

In [10]:
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
import numpy as np
import spacy
from sklearn.base import BaseEstimator, TransformerMixin

class WordVectorTransformer(TransformerMixin,BaseEstimator):
    def __init__(self, model="en_core_web_sm"):
        self.model = model

    def fit(self,X,y=None):
        return self

    def transform(self,X):
        nlp = spacy.load(self.model)
        return np.concatenate([nlp(doc).vector.reshape(1,-1) for doc in X])
# class word2vec():
#     def get_vec(self,x):
#         train_x_vectors = []
#         for x in train_sentence:
#             train_x_vectors.append(nlp(x).vector)

            # yield sparse2full(docvec, len(self.id2word))   
# # Word2Vec
# train_x_vectors = []
# for x in train_sentence:
#      train_x_vectors.append(get_vec(x))

# word2vec = nlp(x).vector

In [ ]:
clf = Pipeline([('tfidf', tfidf), ('clf', classifier)])
clf.fit(train_sentence,train_propaganda)

In [12]:
clf = Pipeline([('word2vec',WordVectorTransformer()), ('clf', classifier)])
clf.fit(train_sentence,train_propaganda)

/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/sklearn/svm/_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


Pipeline(steps=[('word2vec', WordVectorTransformer()), ('clf', LinearSVC())])

In [13]:
y_pred = clf.predict(development_sentence)

print(classification_report(development_propaganda, y_pred))



              precision    recall  f1-score   support

    NEGATIVE       0.80      0.52      0.63      1949
    POSITIVE       0.25      0.56      0.35       567

    accuracy                           0.53      2516
   macro avg       0.53      0.54      0.49      2516
weighted avg       0.68      0.53      0.57      2516

